In [2]:
# from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime
%load_ext line_profiler
pd.options.mode.chained_assignment = None
from datetime import timedelta
import sys
sys.path.insert(0, 'E:/Credence/Projects/Ideal6-python-worker')
import dbUnit as db
import computeUnit as cu

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [11]:
portfolio = db.getDataFromOracle("port_daily","2019-02-01").fillna(0).replace("NA",0)
portfolio['ASONDATE']='2019-02-01'
portfolio['AMOUNT'] = portfolio['AMOUNT'].astype(int)
portWeights = portfolio.pivot_table(index='PORTCODE',columns='ISIN',values="AMOUNT").apply(lambda x: x/(x.sum()),axis=1).stack().reset_index().rename(columns={0:"WEIGHTS"})
# weigthsPort = portfolio.groupby(by="PORTCODE").apply(lambda x:(x['AMOUNT']/x['AMOUNT'].sum())).reset_index().rename(columns={"AMOUNT":"WEIGHTS"}).drop(columns='level_1')
# weigthsPort['ISIN'] = portfolio['ISIN']

portWeights = portfolio.merge(portWeights,on=['PORTCODE',"ISIN"],how='left')
portWeights

,PORTCODE,ISIN,AMOUNT,ASSETCLASS,SECTOR,ASONDATE,WEIGHTS
0,500002,0,1051600,DEBT,49,2019-02-01,-1.425374e-02
1,500002,IN0020009920,100,DEBT,48,2019-02-01,3.260601e-08
2,500002,0,101403,DEBT,2,2019-02-01,-1.425374e-02
3,500002,0,91588148,DEBT,2,2019-02-01,-1.425374e-02
4,500002,0,74146,DEBT,8,2019-02-01,-1.425374e-02
5,500002,0,0,DEBT,5,2019-02-01,-1.425374e-02
6,500002,a,107504368,DEBT,1,2019-02-01,3.505289e-02
7,500002,0,0,DEBT,5,2019-02-01,-1.425374e-02
8,500002,0,50500000,DEBT,5,2019-02-01,-1.425374e-02
9,500002,0,116410,DEPO,1,2019-02-01,-1.425374e-02


In [3]:
# portWeights2 = portfolio.pivot_table(index='PORTCODE',columns='ISIN',values="AMOUNT").apply(lambda x: x/(x.sum()),axis=1)
# finalPortWeights=portWeights2.merge(portWeights2.reindex(groupReturns[~groupReturns.index.isin(portWeights2.columns.values)].index.values,axis=1),left_index=True,right_index=True).fillna(0)[groupReturns.index.values]

In [ ]:
allGroupReturns = db.getDataFromMongo("groupreturns","2018-01-01")
groupReturns = allGroupReturns[allGroupReturns['TYPE']=='SECURITIES'].drop(columns=["ASONDATE","RETURNTYPE","TYPE"]).set_index('CODE')

In [24]:
portReturns = portWeights.merge(groupReturns,left_on='ISIN',right_index=True,how='left')

In [25]:
portReturns[groupReturns.columns] = portReturns[groupReturns.columns].multiply(portReturns['WEIGHTS'],axis='index')

In [26]:
db.insertDataToMongo("portfolioreturns",portReturns,"2018-01-01")

In [27]:
db.insertDataToOracle("portfolioreturns",portReturns)

OperationalError: ORA-03114: not connected to ORACLE

In [28]:
# finalPortWeights.dot(groupReturns)

In [10]:
# pw = portWeights.groupby('PORTCODE').apply(lambda x: ((x["WEIGHTS"].values*groupReturns.transpose()[x['ISIN']].values).transpose()))

# pwgr = pd.DataFrame(np.row_stack(pw))

# pwgr.columns = groupReturns.transpose().index

# pd.concat([portWeights,pwgr],axis=1)

### V2
- Multiply weights with Daily Returns [pct_change()]
- compute log returns
- calculate group returns 1w,1m...

In [ ]:
date = "2019-02-01"
# dailyAdjPrice = pd.read_csv("../../../dailyadjclose_securities.csv",index_col=0)
dailyAdjPrice = db.getDataFromMongo("dailyadjclose_securities",date)

In [129]:
filteredDailyAdjPrice = pd.DataFrame(columns= portWeights['ISIN'].unique())
dailyAdjPrice=filteredDailyAdjPrice.T.merge(dailyAdjPrice.T,left_index=True,right_index=True,how='left').T.replace(np.inf,np.nan).fillna(0)

portfolioDailyAdjPrice = portWeights['WEIGHTS'].values * dailyAdjPrice[portWeights['ISIN'].values]

portfolioLogReturns = np.log(portfolioDailyAdjPrice.pct_change() + 1).replace([np.inf, -np.inf], np.nan).fillna(0)
portfolioLogReturns['ASONDATE']= pd.to_datetime(date, format="%Y-%m-%d")
portfolioLogReturns = portfolioLogReturns.set_index("ASONDATE")

portfolioLogReturns.columns=(portWeights['PORTCODE'].astype(str)+":"+portWeights['ISIN'].astype(str)).values

# dailyLogReturn = pd.read_csv("../../../dailyLogReturn.csv",index_col=0)
# dailyLogReturn = db.getDataFromMongo("dailylogreturns_portsec")

portfolioLogReturns = portfolioLogReturns.T.drop_duplicates().T
if not dailyLogReturn.empty:
    print("notEmpty: getting hist dailyLogReturns from mongo")
    dailyLogReturn = dailyLogReturn[dailyLogReturn['ASONDATE']!=date]
    portfolioLogReturns = dailyLogReturn.append(portfolioLogReturns.reset_index(),sort=False)
    db.insertDataToMongo("dailylogreturns_portsec",portfolioLogReturns)
else:
    print("Empty: saving todays data to mongo")
    db.insertDataToMongo("dailylogreturns_portsec",portfolioLogReturns.reset_index(),date)
    portfolioLogReturns.reset_index(inplace=True)

In [30]:
portLogReturns = {}
portLogReturns['LOGRETURNS'] = portfolioLogReturns.set_index("ASONDATE")
portLogReturns = pd.concat(portLogReturns, axis=1)

In [31]:
groupReturns = cu.calcGroupReturns(date,portLogReturns,["LOGRETURNS"])

groupReturns=groupReturns.reset_index()
groupReturns["PORTCODE"]=groupReturns['level_1'].str.split(":",expand=True)[0].values
groupReturns["ISIN"]=groupReturns['level_1'].str.split(":",expand=True)[1].values
groupReturns.drop(columns=['level_1',"level_0",'PORTCODE'],inplace=True)

In [14]:
portGroupReturns = portWeights.merge(groupReturns,on="ISIN",how="left").drop_duplicates(subset=['PORTCODE','ISIN'],keep='first')

In [17]:
db.insertDataToMongo("portfolioreturns",portGroupReturns,date)